# Optimal Setup for fitting with LPT_RSD

### Based on the tests on abacus simulations described in Maus+(2024) ###

#### 1. Scaling biases with $\sigma_8(z)$ ####
This is a more natural choice that more closely resembles the data space and also helps prevent shifts in marginal $\sigma_8$ constraints to lower values when the data is not sufficiently constraining. We implement this parameterization by sampling in parameters: $(1+b_1)\sigma_8(z)$, $b_2\sigma_8(z)^2$, $b_s\sigma_8(z)^2$, $b_3\sigma_8(z)^3$.
At each step of the MCMC we then convert back to the $(b_1,b_2,b_s,b_3)$ basis to generate the $P_\ell(k)$ theory prediction.

#### 2. Counterterm Parameterization ####

In the default setting in velocileptors the counterterm contribution to $P(k,\mu)$ is given by:
$P_{\rm ct}(k,\mu) = (\alpha_0 + \alpha_2\mu^2 + \alpha_4\mu^4)k^2 P_{\rm s, Zel}(k)$
with $(\alpha_0,\alpha_2,\alpha_4)$ as varied parameters. This parameterization is fine but is not easy to interpret/intuit the values that these $\alpha_i$ can/should take. 

We suggest an alternative parameterization:
$P_{\rm ct}(k,\mu) = (b+f\mu^2)(b\tilde\alpha_0 + f\tilde\alpha_2\mu^2 + f\tilde\alpha_4\mu^4)k^2P_{{\rm s}, b_1^2}(k)$

In this case the counterterm parameters $\tilde\alpha_i$ can be interpreted as "fractional corrections" to linear theory and if we wish for no counterterm to be larger than e.g. 50% of linear theory at a given $k_{\rm max}$ and $\mu$, then we just set priors on $\tilde\alpha_i$ such that $\tilde\alpha_i k_{\rm max}^2 \leq 0.5$. 

This is a more intuitive way for setting priors on the counterterm parameters while not affecting the performance of the model.
This parameterization can be implemented by setting "use_Pzel = False" in the velocileptors.LPT.lpt_rsd_fftw.LPT_RSD() class. Then for a given set of varied parameters $\tilde\alpha_i$, one maps back to the old parameters via:

$\alpha_0 = (1+b_1)^2 \tilde\alpha_0$ 

$\alpha_2 = f(1+b_1)(\tilde\alpha_0+\tilde\alpha_2)$

$\alpha_4 = f(f\tilde\alpha_2+(1+b_1)\tilde\alpha_4)$

$\alpha_6 = f^2\tilde\alpha_4$

#### 3. Scale Cuts ####

We have found that when fitting the monopole and quadrupole moments, the model is stable for k-ranges of $0 \leq k \leq 0.20$ $h$/Mpc and sensitivity to higher-loop effects can cause systematic offsets in cosmological constraints of $\gtrsim 1\sigma$ when $k_{\rm max} \geq 0.22$ $h$/Mpc. When including the hexadecapole moment, nonlinear effects such as Fingers of God can have significant impact on $P_4$. We therefore suggest keeping $k_{4,\rm max} \sim 0.1$ $h$/Mpc for $P_4$, while $k_{\rm max} = 0.2$ is still ok for $P_0$ and $P_2$

#### 4. Priors on Stochastic Terms ####
When data is not constraining enough, the model can suffer from significant parameter volume effects. This problem can be mitigated somewhat with gaussian priors on the stochastic parameters SN$_0$, SN$_2$, SN$_4$ which we can motivate with the 1-halo term of a simple analytic halo model. The expected sizes of the stochastic parameters are roughly:

$SN_0 \sim 1/\bar{n}_g$

$SN_2 \sim f_{sat}\sigma_v^2/\bar{n}_g$

$SN_4 \sim f_{sat}\sigma_v^4/\bar{n}_g$

where $f_{sat}$ is the satellite fraction, $\sigma_v$ is the typical virial velocity dispersion, and $1/\bar{n}_g$ is the galaxy number density of the tracers being modeled.